# Assignemnt-3 KNN

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from tqdm import tqdm
import os


# using the SQLite Table to read data.
con = sqlite3.connect('C:/Users/mlund1/PycharmProjects/Applied AI/Assignment-2/database.sqlite') 



#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, con) 


# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative

In [3]:
print(filtered_data.shape) #looking at the number of attributes and size of the data
print(filtered_data.head())

(525814, 10)
   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
0                     1                       1  positive  1303862400   
1                     0                       0  negative  1346976000   
2                     1                       1  positive  1219017600   
3                     3                       3  negative  1307923200   
4                     0                       0  positive  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several o

In [4]:
print(filtered_data.tail())

            Id   ProductId          UserId              ProfileName  \
525809  568450  B001EO7N10  A28KG5XORO54AY         Lettie D. Carter   
525810  568451  B003S1WTCU  A3I8AFVPEE8KI5                R. Sawyer   
525811  568452  B004I613EE  A121AA1GQV751Z            pksd "pk_007"   
525812  568453  B004I613EE   A3IBEVCTXKNOH  Kathy A. Welch "katwel"   
525813  568454  B001LR2CU2  A3LGQPJCZVL9UC                 srfell17   

        HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
525809                     0                       0  positive  1299628800   
525810                     0                       0  negative  1331251200   
525811                     2                       2  positive  1329782400   
525812                     1                       1  positive  1331596800   
525813                     0                       0  positive  1338422400   

                                   Summary  \
525809                 Will not do without   
525810      

In [5]:
print(filtered_data.isnull().sum()) #Checking for null values


Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64


In [6]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [7]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [8]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [9]:
df_change = final[filtered_data.HelpfulnessNumerator<=filtered_data.HelpfulnessDenominator]
print(df_change.shape)

(364171, 10)


In [10]:
#Checking to see how much % of data still remains
(df_change['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25852107399194

Observation:- It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [11]:
#How many positive & negative Reviews are present in our dataset?
df_change['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

In [12]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [13]:
df_change=df_change[df_change.HelpfulnessNumerator<=df_change.HelpfulnessDenominator]
#Before starting the next phase of preprocessing lets see the number of entries left
df_change.shape


(364171, 10)

In [14]:
#How many positive & negative Reviews are present in our dataset?
df_change['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

In [15]:
#nltk.download('stopwords')
stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer




In [16]:
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'she', 'why', 'so', 'don', 'shouldn', 'd', 'ma', 'me', 'again', 'didn', 'had', 's', 'no', "should've", 'weren', 'mightn', "haven't", 'needn', 'myself', 'll', 'out', "needn't", 'our', 'against', "wasn't", 'i', 'not', 'nor', 'if', 'herself', 'does', "don't", 'yourself', 'you', 'all', 'them', 'both', 'hasn', 'off', 'were', 'for', 'on', 'through', 'its', "doesn't", 'wouldn', 'should', 'they', 'your', 'and', 'then', 'mustn', 'haven', 'of', 'above', "couldn't", 'over', 'most', 'the', 'same', "wouldn't", 'before', 'hers', 'in', 'down', 'but', 'ourselves', 'too', 'after', 'his', 'only', 'theirs', "shouldn't", 'once', 'just', 'aren', "you'll", 'up', "you'd", 'a', 'into', "won't", 'are', 'wasn', 'o', 'y', 'as', 'is', 'when', 'being', 'doesn', 'with', 'other', 'be', 're', 'what', 'any', "aren't", 'below', 'having', 'from', 'whom', "mightn't", 'themselves', 'while', 'can', 'it', 'few', 'ain', 'have', 'shan', 'itself', 'here', 'that', 'my', 'will', 'some', 'himself', 'until', 'has', 'this', 'there

In [17]:
#Code for pre-processing phase where we remove html tag, clean punctutation & special chars,
       
if not os.path.isfile('final.sqlite'):
    i=0
    str1=' '
    final_string=[]
    all_positive_words=[] # store words from +ve reviews here
    all_negative_words=[] # store words from -ve reviews here.
    s=''
    def cleanhtml(sentence): #function to clean the word of any html-tags
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, ' ', sentence)
        return cleantext
    def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
        cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
        cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
        cleaned = re.sub(r'\s*<br\s*/?>\s*',r'',cleaned)
        return  cleaned
    for sent in tqdm(df_change['Text'].values):
        filtered_sentence=[]
        #print(sent);
        sent=cleanhtml(sent) # remove HTMl tags
        for w in sent.split():
            for cleaned_words in cleanpunc(w).split():
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                    if(cleaned_words.lower() not in stop):
                        s=(sno.stem(cleaned_words.lower())).encode('utf8')
                        filtered_sentence.append(s)
                        if (df_change['Score'].values)[i] == 'positive': 
                            all_positive_words.append(s) #list of all words used to describe positive reviews
                        if(df_change['Score'].values)[i] == 'negative':
                            all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                    else:
                        continue
                else:
                    continue 
        #print(filtered_sentence)
        str1 = b" ".join(filtered_sentence) #final string of cleaned words
        #print("***********************************************************************")

        final_string.append(str1)
        i+=1

    

100%|██████████| 364171/364171 [09:51<00:00, 615.38it/s]


In [18]:
df_change['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
df_change['CleanedText']=df_change['CleanedText'].str.decode("utf-8")

In [19]:
df_change.head() #below the processed review can be seen in the CleanedText Column 



,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew read sendak book watch realli rosi movi i...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,fun way children learn month year learn poem t...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,great littl book read nice rhythm well good re...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,book poetri month year goe month cute littl po...


In [20]:
# store final table into an SQlLite table for future.
conn = sqlite3.connect('updatedReviewsdB.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn,  schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [21]:
df_change["Score"].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

In [22]:
# Sorting data based on time
df_change["Time"] = pd.to_datetime(df_change["Time"], unit = "s")
df_change = df_change.sort_values(by = "Time")

Limiting to 30K reviews since machine is running into Memory Error for reviews more than 30 K

In [23]:
df_change["Score"].shape

(364171,)

In [24]:
# To randomly sample 15k points from both class

data_pos = df_change[df_change["Score"] == "positive"].sample(n = 50000)
data_neg = df_change[df_change["Score"] == "negative"].sample(n = 50000)
df_change_100k = pd.concat([data_pos, data_neg])
df_change_100k.shape


(100000, 11)

In [25]:
# To randomly sample 15k points from both class

data_pos = df_change[df_change["Score"] == "positive"].sample(n = 15000)
data_neg = df_change[df_change["Score"] == "negative"].sample(n = 15000)
subset_df_change = pd.concat([data_pos, data_neg])
subset_df_change.shape


(30000, 11)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
#Breaking into Train and test
X_train, X_test, y_train, y_test = train_test_split(subset_df_change['CleanedText'].values,subset_df_change['Score'].values,test_size=0.3,shuffle=False)



# Defining a function to find optimal K


To Find the optimal K we will used 10 fold cross validation method. Based on misclassifiction error for every K, we will decide the best K on Train Data

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

def find_optimal_k(X_train,y_train,algo):
    myList = list(range(0,50))
    neighbors = list(filter(lambda x: x % 2 != 0, myList))
    # empty list that will hold cv scores
    cv_scores = []
    # perform 10-fold cross validation
    for k in tqdm(neighbors):
        knn = KNeighborsClassifier(n_neighbors=k,algorithm=algo)
        scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())

    # changing to misclassification error
    MSE = [1 - x for x in cv_scores]

    # determining best k
    optimal_k = neighbors[MSE.index(min(MSE))]
    print('\nThe optimal number of neighbors is %d.' % optimal_k)


    plt.figure(figsize=(10,6))
    plt.plot(list(filter(lambda x: x % 2 != 0, myList)),MSE,color='blue', linestyle='--', marker= '^',
             markerfacecolor='red', markersize=10)
    plt.title('Error Rate vs. K Value')
    plt.xlabel('Number of Neighbors K')
    plt.ylabel('Error Rate')

    print("the misclassification error for each k value is : ", np.round(MSE,3))
    
    return optimal_k

# BOW

In [ ]:
#Text -> Uni gram Vectors
uni_gram = CountVectorizer() 
X_train = uni_gram.fit_transform(X_train)
#Normalize Data
X_train = preprocessing.normalize(X_train)
print("Train Data Size: ",X_train.shape)
X_test = uni_gram.transform(X_test)
#Normalize Data
X_test = preprocessing.normalize(X_test)
print("Test Data Size: ",X_test.shape)

In [ ]:
print('Original number of features:', X_train.shape[1])
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
standardized_data_sparse_train = csr_matrix(X_train)
standardized_data_sparse_test = csr_matrix(X_test)

tsvd = TruncatedSVD(n_components=10)

X_train_tsvd_train_kd = tsvd.fit(standardized_data_sparse_train).transform(X_train)
X_test_tsvd_test_kd = tsvd.fit(standardized_data_sparse_train).transform(X_test)

print('Reduced number of features:', X_train_tsvd_train_kd.shape[1])

## A. Brute force algo

In [ ]:
# To choose optimal_k using brute force algorithm
optimal_k = find_optimal_k(X_train ,y_train,'brute')


In [ ]:
# ============================== KNN with k = optimal_k ===============================================
# instantiate learning model k = optimal_k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k)
# fitting the model
knn_optimal.fit(X_train, y_train)

In [ ]:
# predict the response
pred = knn_optimal.predict(X_test)


In [ ]:
# evaluate accuracy
acc = accuracy_score(y_test, pred) * 100
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))

# B. kdd_tree algo

In [ ]:
# To choose optimal_k using kdd tree algorithm
optimal_k_kdd_tree = find_optimal_k(X_train_tsvd_train_kd,y_train,'kd_tree')


In [ ]:
# ============================== KNN with k = optimal_k ===============================================
# instantiate learning model k = optimal_k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k_kdd_tree)
# fitting the model
knn_optimal.fit(X_train, y_train)

In [ ]:
# predict the response
pred = knn_optimal.predict(X_test_tsvd_test_kd)


In [ ]:
# evaluate accuracy
acc = accuracy_score(y_test, pred) * 100
print('\nThe accuracy of the knn classifier for k = %d is %f%% using kdd_tree' % (optimal_k, acc))